# 04. 이상 탐지 (Anomaly Detection)

---

## 목차 / Table of Contents
1. 제조 데이터 생성 / Manufacturing Data Generation
2. 통계 기반 이상 탐지 / Statistical Anomaly Detection
3. 지도 학습 기반 이상 탐지 / Supervised Anomaly Detection
4. 임계값 최적화 / Threshold Optimization
5. 이상 샘플 리포트 / Anomaly Report

In [ ]:
# ============================================================
# 라이브러리 임포트 / Library Imports
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 사이킷런 / Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    precision_recall_curve
)

# XGBoost
from xgboost import XGBClassifier

# 불균형 처리 / Imbalanced data
from imblearn.over_sampling import SMOTE

# 설정 / Settings
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

print("✅ 라이브러리 로드 완료 / Libraries loaded")

---
## 1. 제조 데이터 생성 / Manufacturing Data Generation

제조 라인 이상 탐지 문제를 시뮬레이션합니다.
Simulate a manufacturing line anomaly detection problem.

In [ ]:
# ============================================================
# 제조 데이터 생성 / Create Manufacturing Data
# ============================================================

def create_manufacturing_data(n_samples=2000, n_sensors=15, anomaly_ratio=0.05, random_state=42):
    """
    제조 이상 탐지용 데이터 생성
    Create manufacturing anomaly detection data
    
    센서 데이터 특성:
    - 온도, 압력, 진동 등 다양한 센서
    - 이상 시 특정 센서들의 값이 비정상적으로 변함
    """
    np.random.seed(random_state)
    
    n_normal = int(n_samples * (1 - anomaly_ratio))
    n_anomaly = n_samples - n_normal
    
    # 센서 이름 / Sensor names
    sensor_names = [
        'temp_1', 'temp_2', 'temp_3',
        'pressure_1', 'pressure_2',
        'vibration_1', 'vibration_2', 'vibration_3',
        'speed_1', 'speed_2',
        'current_1', 'current_2',
        'humidity', 'flow_rate', 'power'
    ]
    
    # 정상 데이터 생성 / Generate normal data
    normal_data = {}
    # 온도 센서 (35-45도 범위)
    normal_data['temp_1'] = np.random.normal(40, 2, n_normal)
    normal_data['temp_2'] = np.random.normal(42, 2, n_normal)
    normal_data['temp_3'] = np.random.normal(38, 2, n_normal)
    # 압력 센서 (100-120 범위)
    normal_data['pressure_1'] = np.random.normal(110, 5, n_normal)
    normal_data['pressure_2'] = np.random.normal(108, 5, n_normal)
    # 진동 센서 (0.1-0.5 범위)
    normal_data['vibration_1'] = np.random.normal(0.3, 0.05, n_normal)
    normal_data['vibration_2'] = np.random.normal(0.28, 0.05, n_normal)
    normal_data['vibration_3'] = np.random.normal(0.32, 0.05, n_normal)
    # 기타 센서
    normal_data['speed_1'] = np.random.normal(1500, 50, n_normal)
    normal_data['speed_2'] = np.random.normal(1480, 50, n_normal)
    normal_data['current_1'] = np.random.normal(15, 1, n_normal)
    normal_data['current_2'] = np.random.normal(14.5, 1, n_normal)
    normal_data['humidity'] = np.random.normal(45, 5, n_normal)
    normal_data['flow_rate'] = np.random.normal(100, 10, n_normal)
    normal_data['power'] = np.random.normal(500, 20, n_normal)
    
    X_normal = pd.DataFrame(normal_data)
    y_normal = np.zeros(n_normal)
    
    # 이상 데이터 생성 / Generate anomaly data
    anomaly_data = {}
    # 이상 시: 온도 상승, 진동 증가, 전류 증가
    anomaly_data['temp_1'] = np.random.normal(48, 3, n_anomaly)  # 높은 온도
    anomaly_data['temp_2'] = np.random.normal(50, 3, n_anomaly)
    anomaly_data['temp_3'] = np.random.normal(45, 3, n_anomaly)
    anomaly_data['pressure_1'] = np.random.normal(95, 8, n_anomaly)  # 불안정한 압력
    anomaly_data['pressure_2'] = np.random.normal(125, 8, n_anomaly)
    anomaly_data['vibration_1'] = np.random.normal(0.6, 0.1, n_anomaly)  # 높은 진동
    anomaly_data['vibration_2'] = np.random.normal(0.55, 0.1, n_anomaly)
    anomaly_data['vibration_3'] = np.random.normal(0.58, 0.1, n_anomaly)
    anomaly_data['speed_1'] = np.random.normal(1400, 80, n_anomaly)  # 불안정한 속도
    anomaly_data['speed_2'] = np.random.normal(1380, 80, n_anomaly)
    anomaly_data['current_1'] = np.random.normal(18, 2, n_anomaly)  # 높은 전류
    anomaly_data['current_2'] = np.random.normal(17, 2, n_anomaly)
    anomaly_data['humidity'] = np.random.normal(55, 8, n_anomaly)
    anomaly_data['flow_rate'] = np.random.normal(85, 15, n_anomaly)
    anomaly_data['power'] = np.random.normal(580, 30, n_anomaly)
    
    X_anomaly = pd.DataFrame(anomaly_data)
    y_anomaly = np.ones(n_anomaly)
    
    # 합치기 / Combine
    X = pd.concat([X_normal, X_anomaly], ignore_index=True)
    y = np.hstack([y_normal, y_anomaly])
    
    # 셔플 / Shuffle
    shuffle_idx = np.random.permutation(len(X))
    X = X.iloc[shuffle_idx].reset_index(drop=True)
    y = y[shuffle_idx]
    
    # 일부 결측치 추가 / Add some missing values
    mask = np.random.random(X.shape) < 0.01
    X = X.mask(mask)
    
    return X, y

# 데이터 생성 / Generate data
X, y = create_manufacturing_data(n_samples=2000, anomaly_ratio=0.05)

print(f"데이터 shape: {X.shape}")
print(f"\n클래스 분포 / Class distribution:")
print(f"  정상 (0): {(y == 0).sum()}")
print(f"  이상 (1): {(y == 1).sum()}")
print(f"  이상 비율: {y.mean():.2%}")

X.head()

In [ ]:
# ============================================================
# 데이터 탐색 / Data Exploration
# ============================================================

# 센서별 분포 비교 / Compare sensor distributions by class
fig, axes = plt.subplots(3, 5, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(X.columns):
    # 정상 vs 이상 분포 / Normal vs Anomaly distribution
    axes[i].hist(X.loc[y == 0, col].dropna(), bins=30, alpha=0.5, label='Normal', color='blue')
    axes[i].hist(X.loc[y == 1, col].dropna(), bins=30, alpha=0.5, label='Anomaly', color='red')
    axes[i].set_title(col)
    axes[i].legend(fontsize=8)

plt.tight_layout()
plt.suptitle('Sensor Distributions by Class / 클래스별 센서 분포', y=1.02, fontsize=14)
plt.show()

In [ ]:
# ============================================================
# 데이터 전처리 / Data Preprocessing
# ============================================================

# 결측치 처리 / Handle missing values
print(f"결측치 수: {X.isnull().sum().sum()}")
X_filled = X.fillna(X.mean())

# Train/Test 분할 (층화 샘플링) / Train/Test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X_filled, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# 스케일링 / Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n훈련 세트: {X_train.shape}, 이상 비율: {y_train.mean():.2%}")
print(f"테스트 세트: {X_test.shape}, 이상 비율: {y_test.mean():.2%}")

---
## 2. 통계 기반 이상 탐지 / Statistical Anomaly Detection

In [ ]:
# ============================================================
# Z-score 기반 이상 탐지 / Z-score based Anomaly Detection
# ============================================================

def detect_anomaly_zscore(X, threshold=3):
    """
    Z-score 기반 이상 탐지
    Z-score based anomaly detection
    
    어떤 센서라도 Z-score가 threshold를 넘으면 이상으로 판정
    Mark as anomaly if any sensor's Z-score exceeds threshold
    """
    # Z-score 계산 / Calculate Z-scores
    z_scores = np.abs((X - X.mean()) / X.std())
    
    # 어떤 센서라도 threshold 초과하면 이상 / Anomaly if any sensor exceeds threshold
    is_anomaly = (z_scores > threshold).any(axis=1).astype(int)
    
    return is_anomaly

# Z-score 이상 탐지 / Z-score anomaly detection
y_pred_zscore = detect_anomaly_zscore(X_test, threshold=3)

print("Z-score 이상 탐지 결과:")
print(f"  탐지된 이상: {y_pred_zscore.sum()}")
print(f"  실제 이상: {int(y_test.sum())}")
print(f"\n분류 보고서:")
print(classification_report(y_test, y_pred_zscore, target_names=['Normal', 'Anomaly']))

In [ ]:
# ============================================================
# Isolation Forest (비지도 이상 탐지) / Isolation Forest (Unsupervised)
# ============================================================

# Isolation Forest 모델 / Isolation Forest model
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,  # 예상 이상 비율 / Expected anomaly ratio
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# 학습 및 예측 / Train and predict
iso_forest.fit(X_train_scaled)
y_pred_iso = iso_forest.predict(X_test_scaled)

# -1 = 이상, 1 = 정상 → 0/1로 변환
# -1 = anomaly, 1 = normal → convert to 0/1
y_pred_iso = (y_pred_iso == -1).astype(int)

print("Isolation Forest 이상 탐지 결과:")
print(f"  탐지된 이상: {y_pred_iso.sum()}")
print(f"  실제 이상: {int(y_test.sum())}")
print(f"\n분류 보고서:")
print(classification_report(y_test, y_pred_iso, target_names=['Normal', 'Anomaly']))

---
## 3. 지도 학습 기반 이상 탐지 / Supervised Anomaly Detection

레이블이 있을 때 분류 모델을 사용한 이상 탐지입니다.

In [ ]:
# ============================================================
# 불균형 처리: SMOTE / Handle Imbalance: SMOTE
# ============================================================

print("SMOTE 적용 전:")
print(f"  Class 0 (정상): {(y_train == 0).sum()}")
print(f"  Class 1 (이상): {(y_train == 1).sum()}")

# SMOTE 적용 / Apply SMOTE
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("\nSMOTE 적용 후:")
print(f"  Class 0 (정상): {(y_train_smote == 0).sum()}")
print(f"  Class 1 (이상): {(y_train_smote == 1).sum()}")

In [ ]:
# ============================================================
# Random Forest 이상 탐지 / Random Forest Anomaly Detection
# ============================================================

# 모델 학습 / Train model
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf_model.fit(X_train_scaled, y_train)

# 예측 / Predict
y_pred_rf = rf_model.predict(X_test_scaled)
y_prob_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

print("Random Forest 이상 탐지 결과:")
print(classification_report(y_test, y_pred_rf, target_names=['Normal', 'Anomaly']))
print(f"\nROC-AUC: {roc_auc_score(y_test, y_prob_rf):.4f}")

In [ ]:
# ============================================================
# XGBoost 이상 탐지 / XGBoost Anomaly Detection
# ============================================================

# scale_pos_weight 계산 / Calculate scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# 모델 학습 / Train model
xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_STATE,
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=-1
)

xgb_model.fit(X_train_scaled, y_train)

# 예측 / Predict
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_prob_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

print("XGBoost 이상 탐지 결과:")
print(classification_report(y_test, y_pred_xgb, target_names=['Normal', 'Anomaly']))
print(f"\nROC-AUC: {roc_auc_score(y_test, y_prob_xgb):.4f}")

In [ ]:
# ============================================================
# XGBoost (SMOTE) 이상 탐지 / XGBoost with SMOTE
# ============================================================

xgb_smote = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=-1
)

# SMOTE 데이터로 학습 / Train with SMOTE data
xgb_smote.fit(X_train_smote, y_train_smote)

# 예측 / Predict
y_pred_xgb_smote = xgb_smote.predict(X_test_scaled)
y_prob_xgb_smote = xgb_smote.predict_proba(X_test_scaled)[:, 1]

print("XGBoost (SMOTE) 이상 탐지 결과:")
print(classification_report(y_test, y_pred_xgb_smote, target_names=['Normal', 'Anomaly']))
print(f"\nROC-AUC: {roc_auc_score(y_test, y_prob_xgb_smote):.4f}")

---
## 4. 임계값 최적화 / Threshold Optimization

In [ ]:
# ============================================================
# 최적 임계값 탐색 / Find Optimal Threshold
# ============================================================

def find_optimal_threshold(y_true, y_prob, metric='f1'):
    """
    최적 임계값 찾기
    Find optimal threshold
    
    이상 탐지에서는 Recall(미탐 최소화)이 중요할 수 있음
    For anomaly detection, Recall (minimize FN) may be more important
    """
    thresholds = np.arange(0.1, 0.9, 0.01)
    results = []
    
    for thresh in thresholds:
        y_pred = (y_prob >= thresh).astype(int)
        
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        
        results.append({
            'threshold': thresh,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    results_df = pd.DataFrame(results)
    best_idx = results_df[metric].idxmax()
    best_threshold = results_df.loc[best_idx, 'threshold']
    
    return best_threshold, results_df

# 최적 임계값 찾기 (F1 기준) / Find optimal threshold (based on F1)
best_thresh_f1, thresh_results = find_optimal_threshold(y_test, y_prob_xgb_smote, 'f1')
print(f"최적 임계값 (F1 기준): {best_thresh_f1:.2f}")

# 최적 임계값 찾기 (Recall 기준) / Find optimal threshold (based on Recall)
best_thresh_recall, _ = find_optimal_threshold(y_test, y_prob_xgb_smote, 'recall')
print(f"최적 임계값 (Recall 기준): {best_thresh_recall:.2f}")

In [ ]:
# ============================================================
# 임계값별 성능 시각화 / Visualize Performance by Threshold
# ============================================================

plt.figure(figsize=(12, 5))

plt.plot(thresh_results['threshold'], thresh_results['precision'], 'b-', label='Precision')
plt.plot(thresh_results['threshold'], thresh_results['recall'], 'r-', label='Recall')
plt.plot(thresh_results['threshold'], thresh_results['f1'], 'g-', label='F1 Score', linewidth=2)
plt.axvline(best_thresh_f1, color='green', linestyle='--', alpha=0.7, label=f'Best F1 ({best_thresh_f1:.2f})')
plt.axvline(0.5, color='gray', linestyle=':', alpha=0.5, label='Default (0.5)')

plt.xlabel('Threshold / 임계값')
plt.ylabel('Score')
plt.title('Performance by Threshold / 임계값별 성능')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 최적 임계값 적용 / Apply Optimal Threshold
# ============================================================

print("=" * 60)
print("📊 임계값 비교 / Threshold Comparison")
print("=" * 60)

# 기본 임계값 (0.5)
print("\n[기본 임계값 0.5 / Default threshold 0.5]")
print(classification_report(y_test, y_pred_xgb_smote, target_names=['Normal', 'Anomaly']))

# 최적 임계값
print(f"\n[최적 임계값 {best_thresh_f1:.2f} / Optimal threshold {best_thresh_f1:.2f}]")
y_pred_optimal = (y_prob_xgb_smote >= best_thresh_f1).astype(int)
print(classification_report(y_test, y_pred_optimal, target_names=['Normal', 'Anomaly']))

---
## 5. 이상 샘플 리포트 / Anomaly Report

In [ ]:
# ============================================================
# 피처 중요도 / Feature Importance
# ============================================================

# XGBoost 피처 중요도
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_smote.feature_importances_
}).sort_values('importance', ascending=False)

print("피처 중요도 (이상 탐지 기여도):")
print(importance_df)

# 시각화 / Visualize
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'], color='coral')
plt.xlabel('Importance')
plt.title('Feature Importance for Anomaly Detection / 이상 탐지 피처 중요도')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 이상 샘플 리포트 생성 / Generate Anomaly Report
# ============================================================

def generate_anomaly_report(X_test, y_test, y_pred, y_prob, top_n=10):
    """
    이상 샘플 리포트 생성
    Generate anomaly sample report
    """
    # 결과 DataFrame 생성 / Create result DataFrame
    report_df = X_test.copy()
    report_df['actual'] = y_test
    report_df['predicted'] = y_pred
    report_df['anomaly_prob'] = y_prob
    
    # 분류 결과 / Classification result
    conditions = [
        (report_df['actual'] == 1) & (report_df['predicted'] == 1),  # TP
        (report_df['actual'] == 0) & (report_df['predicted'] == 1),  # FP
        (report_df['actual'] == 1) & (report_df['predicted'] == 0),  # FN
        (report_df['actual'] == 0) & (report_df['predicted'] == 0),  # TN
    ]
    choices = ['TP (정탐)', 'FP (오탐)', 'FN (미탐)', 'TN (정상)']
    report_df['result'] = np.select(conditions, choices)
    
    return report_df

# 리포트 생성 / Generate report
report = generate_anomaly_report(X_test.reset_index(drop=True), y_test, y_pred_optimal, y_prob_xgb_smote)

print("분류 결과 요약 / Classification Result Summary:")
print(report['result'].value_counts())

In [ ]:
# ============================================================
# 이상 확률 상위 샘플 / Top Anomaly Probability Samples
# ============================================================

# 이상 확률 상위 10개 / Top 10 by anomaly probability
top_anomalies = report.nlargest(10, 'anomaly_prob')[['anomaly_prob', 'actual', 'predicted', 'result']]

print("\n이상 확률 상위 10개 샘플 / Top 10 Anomaly Probability Samples:")
print(top_anomalies)

In [ ]:
# ============================================================
# 미탐(FN) 분석 / False Negative Analysis
# ============================================================

# 미탐된 이상 샘플 / False negative samples
fn_samples = report[report['result'] == 'FN (미탐)']

if len(fn_samples) > 0:
    print(f"미탐(FN) 샘플 수: {len(fn_samples)}")
    print("\n미탐 샘플 특성 (평균):")
    
    # 정상/이상/미탐 비교 / Compare normal/anomaly/FN
    comparison = pd.DataFrame({
        'Normal Mean': report[report['actual'] == 0][X.columns].mean(),
        'Anomaly Mean': report[report['actual'] == 1][X.columns].mean(),
        'FN Mean': fn_samples[X.columns].mean()
    })
    print(comparison.round(2))
else:
    print("✅ 미탐(FN)이 없습니다! / No false negatives!")

In [ ]:
# ============================================================
# 최종 혼동 행렬 시각화 / Final Confusion Matrix Visualization
# ============================================================

cm = confusion_matrix(y_test, y_pred_optimal)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal (0)', 'Anomaly (1)'],
            yticklabels=['Normal (0)', 'Anomaly (1)'])
plt.xlabel('Predicted / 예측')
plt.ylabel('Actual / 실제')
plt.title(f'Confusion Matrix (Threshold={best_thresh_f1:.2f})')

# 주석 추가 / Add annotations
plt.text(0.5, -0.1, f"TN={cm[0,0]}, FP={cm[0,1]}, FN={cm[1,0]}, TP={cm[1,1]}", 
         ha='center', transform=plt.gca().transAxes, fontsize=10)

plt.tight_layout()
plt.show()

---
## 요약: 이상 탐지 체크리스트 / Summary: Anomaly Detection Checklist

```python
# 1. 데이터 탐색
# - 클래스 분포 확인 (불균형 정도)
# - 센서별 정상/이상 분포 비교

# 2. 전처리
X_filled = X.fillna(X.mean())
X_train_scaled = scaler.fit_transform(X_train)

# 3. 불균형 처리 선택
# 방법 A: class_weight / scale_pos_weight
# 방법 B: SMOTE

# 4. 모델 학습
xgb_model = XGBClassifier(scale_pos_weight=ratio)
xgb_model.fit(X_train_scaled, y_train)

# 5. 임계값 최적화
best_thresh, _ = find_optimal_threshold(y_test, y_prob, 'f1')
y_pred = (y_prob >= best_thresh).astype(int)

# 6. 평가 (F1, Recall 중요)
print(classification_report(y_test, y_pred))

# 7. 피처 중요도 분석
importance = model.feature_importances_
```

In [ ]:
print("✅ 이상 탐지 노트북 완료 / Anomaly Detection notebook complete!")
print("\n다음 노트북: 05_deep_learning.ipynb")
print("Next notebook: 05_deep_learning.ipynb")